# Ocean Heat Content

Regridding the high resolution year averaged files was performed with `../regrid/regrid_yrly_TEMP_PD.py`.

Here the lowres and regridded hires TEMP field will be pointwise quadratially detrended.

Calculating the OHC file is done via 

```
from ac_derivation_OHC import DeriveOHC as DO
DO().generate_OHC_files('ctrl', year=y)
DO().combine_yrly_OHC_integral_files(run='ctrl', pwqd=True)
```

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from paths import CESM_filename, path_samoc
from timeseries import IterateOutputCESM
from xr_regression import xr_quadtrend
from aa_derivation_fields import DeriveField as DF
from dask.distributed import Client, LocalCluster
from ac_derivation_OHC import DeriveOHC as DO

In [ ]:
cluster = LocalCluster(n_workers=2)
client = Client(cluster)

need to execute:
`ssh -N -L 8787:127.0.0.1:8787 dijkbio@int2-bb.cartesius.surfsara.nl`

In [ ]:
%%time
# 12:39 mins
DF('ctrl').make_pwqd_TEMP_files()

In [ ]:
%%time
DO().generate_OHC_files('lpd', pwqd=True)
DO().combine_yrly_OHC_integral_files(run='lpd', pwqd=True)

In [ ]:
for y,m,file in IterateOutputCESM(domain='ocn_rect', run='ctrl', tavg='yrly', name='TEMP_pwqd'):
    assert os.path.exists(file)

In [ ]:
da = xr.open_dataset(f'{path_samoc}/ctrl_rect/TEMP_PD_yrly_0274.interp900x602.nc')

In [ ]:
da.TEMP[0,:,:].plot()

In [ ]:
da = xr.open_dataset(f'{path_samoc}/ctrl_rect/TEMP_pwqd_yrly_0278.interp900x602.nc')

In [ ]:
da.TEMP[0,:,:].plot()

In [ ]:
da = xr.open_dataset(f'{path_samoc}/ctrl_rect/TEMP_yrly.interp900x602.nc')

In [ ]:
da.TEMP[:,0,200,200].plot()

In [ ]:
ds = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc')


In [ ]:
plt.plot(ds.time/365, ds.OHC_Global_Ocean)

In [ ]:
ds = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_pwqd.nc')
ds_ = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc').isel(time=slice(50,300))

In [ ]:
plt.plot(ds.time/365, ds.OHC_Global_Ocean)
plt.plot(ds_.time/365, (ds_.OHC_Global_Ocean-xr_quadtrend(ds_.OHC_Global_Ocean)))


In [ ]:
da_ = xr.open_mfdataset(f'{path_samoc}/ctrl_rect/TEMP_pwqd_yrly_*.interp900x602.nc',
                        concat_dim=['time'], chunks={'depth_t':1})

In [ ]:
from dask_jobqueue import SLURMCluster

In [ ]:
cluster = SLURMCluster(processes=6, cores=24, memory="24GB", queue="normal", walltime="01:00:00")

In [ ]:
cluster.scale(10)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
# cluster = LocalCluster(n_workers=2)
# client = Client(cluster)

In [ ]:
%%time
# 13min 37sec
DF('lpd').make_pwqd_TEMP_files()

In [ ]:
da = xr.open_dataarray(f'{path_samoc}/ctrl_rect/TEMP_yrly.interp900x602.nc', decode_times=False)